In [33]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import pickle
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv


In [34]:
# Load environment variables
load_dotenv()

# Database connection configuration
# DB_USER = os.getenv('DB_USER')
# DB_PASSWORD = os.getenv('DB_PASSWORD')
# DB_HOST = os.getenv('DB_HOST', 'localhost')
# DB_PORT = os.getenv('DB_PORT', '3306')
# DB_NAME = 'game_analytics'

DB_USER='root'
DB_PASSWORD='root'
DB_HOST='localhost'
DB_PORT=3306
DB_NAME='game_analytics'


In [35]:
# Step 1: Create database connection and fetch data
print("Step 1: Connecting to database and fetching data...")
try:
    # Create SQLAlchemy engine
    database_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(database_url)
    
    # Fetch data using SQL query
    query = """
    SELECT 
        BIN_TO_UUID(pgs.player_id) as player_id,
        pgs.game_name,
        pgs.total_time_played_minutes,
        pgs.total_games_played,
        pgs.total_wins,
        pgs.total_moves,
        pgs.age  -- Assuming there's a players table with age
    FROM 
        player_game_stats pgs
        JOIN players p ON pgs.player_id = p.player_id
    WHERE 
        pgs.total_games_played > 0  -- Filter out players with no games
        AND pgs.total_time_played_minutes > 0;
    """
    
    df = pd.read_sql(query, engine)
    print(f"Fetched {len(df)} records from database")
    
except Exception as e:
    print(f"Error connecting to database: {str(e)}")
    raise

Step 1: Connecting to database and fetching data...
Fetched 1000 records from database


In [36]:
# Step 2: Basic feature engineering
print("Step 2: Creating basic features...")
df['avg_session_duration'] = df['total_time_played_minutes'] / df['total_games_played']
df['historical_win_rate'] = df['total_wins'] / df['total_games_played']
df['avg_moves_per_game'] = df['total_moves'] / df['total_games_played']

Step 2: Creating basic features...


In [37]:
# Step 3: Encode game names into numbers
print("Step 3: Encoding game names...")
game_encoder = LabelEncoder()
df['game_encoded'] = game_encoder.fit_transform(df['game_name'])

Step 3: Encoding game names...


In [38]:
# Step 4: Calculate win rate per game type
print("Step 4: Calculating game-specific win rates...")
game_win_rates = df.groupby('game_name')['historical_win_rate'].mean().reset_index()
game_win_rates.columns = ['game_name', 'game_avg_win_rate']
df = df.merge(game_win_rates, on='game_name')
display(df.head(500))

Step 4: Calculating game-specific win rates...


,player_id,game_name,total_time_played_minutes,total_games_played,total_wins,total_moves,age,avg_session_duration,historical_win_rate,avg_moves_per_game,game_encoded,game_avg_win_rate
0,d16771fa-ba60-11ef-9d4e-00155d326b4e,battleship,5990,170,92,4914,34,35.235294,0.541176,28.905882,0,0.500907
1,d133c41a-ba60-11ef-9d4e-00155d326b4e,battleship,5094,148,76,4220,18,34.418919,0.513514,28.513514,0,0.500907
2,d15034ff-ba60-11ef-9d4e-00155d326b4e,battleship,7171,210,105,6053,51,34.147619,0.500000,28.823810,0,0.500907
3,d15ee3cd-ba60-11ef-9d4e-00155d326b4e,battleship,4285,119,61,3471,33,36.008403,0.512605,29.168067,0,0.500907
4,d1224a65-ba60-11ef-9d4e-00155d326b4e,battleship,4969,147,79,4230,18,33.802721,0.537415,28.775510,0,0.500907
...,...,...,...,...,...,...,...,...,...,...,...,...
495,d163fffc-ba60-11ef-9d4e-00155d326b4e,connect four,1273,36,17,951,45,35.361111,0.472222,26.416667,2,0.464285
496,d0f948f1-ba60-11ef-9d4e-00155d326b4e,connect four,380,12,6,469,47,31.666667,0.500000,39.083333,2,0.464285
497,d1633018-ba60-11ef-9d4e-00155d326b4e,connect four,1066,35,15,915,42,30.457143,0.428571,26.142857,2,0.464285
498,d0d1a7d7-ba60-11ef-9d4e-00155d326b4e,connect four,462,15,8,431,35,30.800000,0.533333,28.733333,2,0.464285


In [39]:
# Print summary statistics
print("\nData Summary:")
display(f"Number of unique players: {df['player_id'].nunique()}")
display(f"Number of unique games: {df['game_name'].nunique()}")
display("\nGames distribution:")
display(df['game_name'].value_counts())
display("\nFeature statistics:")
display(df[['avg_session_duration', 'historical_win_rate', 'avg_moves_per_game']].describe())


Data Summary:


'Number of unique players: 200'

'Number of unique games: 5'

'\nGames distribution:'

game_name
battleship        200
chess             200
connect four      200
tic tac toe       200
dots and boxes    200
Name: count, dtype: int64

'\nFeature statistics:'

,avg_session_duration,historical_win_rate,avg_moves_per_game
count,1000.000000,1000.000000,1000.000000
mean,34.485636,0.475488,29.503620
std,3.239447,0.112603,2.588481
min,17.571429,0.100000,20.083333
25%,32.749188,0.408931,28.106061
50%,34.419740,0.475223,29.550505
75%,36.235229,0.539429,30.912990
max,47.000000,1.000000,40.600000


In [40]:
# Step 5: Select features for model
print("\nStep 5: Preparing features for model...")
features = [
    'avg_session_duration',
    'historical_win_rate',
    'avg_moves_per_game',
    'game_encoded',
    'game_avg_win_rate',
    'age'
]

X = df[features]



Step 5: Preparing features for model...


In [41]:
# Step 6: Create target variable (win or lose)
print("Step 6: Creating target variable...")
df['next_game_win'] = (df['historical_win_rate'] > df['historical_win_rate'].mean()).astype(int)
y = df['next_game_win']


Step 6: Creating target variable...


In [42]:
# Step 7: Split data into train and test sets
print("Step 7: Splitting data...")
unique_players = df['player_id'].unique()
train_players, test_players = train_test_split(unique_players, test_size=0.2, random_state=42)

train_mask = df['player_id'].isin(train_players)
test_mask = df['player_id'].isin(test_players)

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]


Step 7: Splitting data...


In [43]:
# Step 8: Scale the features
print("Step 8: Scaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Step 8: Scaling features...


In [44]:
# Step 9: Train the model
print("Step 9: Training model...")
model = GradientBoostingClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

Step 9: Training model...


GradientBoostingClassifier(random_state=42)

In [45]:
# Step 10: Evaluate the model
print("Step 10: Evaluating model...")
y_pred = model.predict(X_test_scaled)
print("\nModel Performance:")
print(classification_report(y_test, y_pred))

Step 10: Evaluating model...

Model Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        95
           1       1.00      1.00      1.00       105

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [46]:
# Step 11: Show feature importance
print("\nFeature Importance:")
importance_df = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print(importance_df)


Feature Importance:
                feature    importance
1   historical_win_rate  1.000000e+00
2    avg_moves_per_game  5.731532e-15
0  avg_session_duration  2.000844e-15
5                   age  6.351569e-16
4     game_avg_win_rate  4.855031e-18
3          game_encoded  0.000000e+00


In [47]:
# # Step 12: Save the model and necessary components
# print("\nStep 12: Saving model and components...")
# model_dir = 'model'
# os.makedirs(model_dir, exist_ok=True)
# 
# with open(f'{model_dir}/win_probability_model.pkl', 'wb') as f:
#     pickle.dump(model, f)
# with open(f'{model_dir}/win_probability_scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)
# with open(f'{model_dir}/game_encoder.pkl', 'wb') as f:
#     pickle.dump(game_encoder, f)
# 
# # Save feature importance for reference
# importance_df.to_csv(f'{model_dir}/feature_importance.csv', index=False)

print("\nTraining complete! Model and components saved in 'models' directory.")


Training complete! Model and components saved in 'models' directory.


In [48]:
# Close database connection
engine.dispose()